# Fast re-run training for all models (baseline k=1)
This notebook re-trains **Taylor, Modified Taylor, Discretion, Commitment** using a faster CPU-friendly plan.
It keeps the *same equations/DEQN algorithm* but reduces the most expensive knobs (steps/val size/replay buffer) to speed up.

Outputs: run directories in `../artifacts/runs/<policy>/...` and updates `../artifacts/selected_runs.json`.


In [ ]:

import os, sys, json, numpy as np, torch
sys.path.append("..")
from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer
from src.steady_states import solve_flexprice_sss, export_rbar_tensor, solve_taylor_sss, solve_discretion_sss, solve_commitment_sss
from src.io_utils import make_run_dir, save_run_metadata, save_selected_run

ARTIFACTS = "../artifacts"
device="cpu"
dtype=torch.float32
params = ModelParams(device=device, dtype=dtype)  # baseline: sigma_*_bad_mult=1


## Fast training presets
Two stages:
- Stage A: gh_n=2 (cheap) to get close
- Stage B: gh_n=3 (paper baseline) to refine

You can tweak `STEPS_A/ B` if you want more accuracy.

In [ ]:

# --- global knobs ---
STEPS_A = 4000
STEPS_B = 8000

# Validation reduced for speed (still useful)
VAL_SIZE = 4096
VAL_EVERY = 400

# Replay buffer reduced for RAM/speed
RB_SIZE = 120_000
REFRESH_EVERY = 80
REFRESH_STEPS = 60
INIT_FILL = 300

def cfg_stage(run_dir, *, seed, batch, lr, steps, gh_n, early_stop=False, patience=3000):
    return TrainConfig(
        seed=seed,
        steps=steps,
        batch_size=batch,
        lr=lr,
        grad_clip=1.0,
        gh_n_train=gh_n,
        replay_buffer_size=RB_SIZE,
        replay_refresh_every=REFRESH_EVERY,
        replay_refresh_steps=REFRESH_STEPS,
        replay_init_fill_steps=INIT_FILL if steps==STEPS_A else 0,
        val_size=VAL_SIZE,
        val_every=VAL_EVERY,
        early_stopping=early_stop,
        patience=patience,
        artifacts_root=ARTIFACTS,
        run_dir=run_dir,
        save_best=True,
        enable_timers=True,
        log_every=50,
        profile=False,
        matmul_precision="high",
    )


## Helper: train one policy

In [ ]:

def train_policy(policy: str, d_in: int, d_out: int, *, seed=123, tag="project1_fast"):
    run_dir = make_run_dir(ARTIFACTS, policy=policy, tag=tag, seed=seed)
    cfgA = cfg_stage(run_dir, seed=seed, batch=256 if policy in ["taylor","mod_taylor"] else 96, lr=1e-5, steps=STEPS_A, gh_n=2)
    cfgB = cfg_stage(run_dir, seed=seed, batch=256 if policy in ["taylor","mod_taylor"] else 96, lr=1e-5, steps=STEPS_B, gh_n=3,
                     early_stop=(policy=="commitment"), patience=3000)

    # steady states
    flex = solve_flexprice_sss(params)
    rbar = export_rbar_tensor(flex, params)
    if policy in ["taylor","mod_taylor"]:
        sss = solve_taylor_sss(params)  # includes Delta_ss etc
        commitment_sss=None
    elif policy=="discretion":
        sss = solve_discretion_sss(params)
        commitment_sss=None
    else:
        sss = solve_commitment_sss(params)
        commitment_sss=sss["commitment_init_multipliers"]

    net = PolicyNetwork(d_in=d_in, d_out=d_out, hidden=cfgA.hidden_layers, activation=cfgA.activation)
    tr = Trainer(params=params, cfg=cfgA, policy=policy, net=net, rbar_by_regime=(rbar if policy=="mod_taylor" else None), gh_n=cfgA.gh_n_train)
    save_run_metadata(run_dir, {"policy":policy, "stage":"A", **cfgA.__dict__, "model_params": params.__dict__}, notes="fast stage A")

    tr.train(sss=sss, commitment_sss=commitment_sss)
    # stage B refine
    tr.cfg = cfgB
    tr.gh_n = cfgB.gh_n_train
    tr._eps_grid, tr._w_grid = None, None  # force recache
    tr.__post_init__()
    save_run_metadata(run_dir, {"policy":policy, "stage":"B", **cfgB.__dict__, "model_params": params.__dict__}, notes="fast stage B")
    tr.train(sss=sss, commitment_sss=commitment_sss)

    # mark as selected
    save_selected_run(ARTIFACTS, policy=policy, run_dir=run_dir)
    return run_dir


## Train all (baseline k=1)

In [ ]:

# Dimensions (from our spec)
dims = {
    "taylor": (5,8),
    "mod_taylor": (5,8),
    "discretion": (5,11),
    "commitment": (7,13),
}
run_dirs={}
for pol,(d_in,d_out) in dims.items():
    print("Training", pol)
    run_dirs[pol]=train_policy(pol, d_in, d_out)

run_dirs
